# Demo 0: KX2 & SCILA & Byonoy Routine

## Import Statemets

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import asyncio
from tqdm.auto import tqdm

## SCILA Incubator Setup

1. import modules you require
2. create a SCILA incubator backend, and give it the device's IP address
3. call `.setup()` to establish a connection and initialize the device
4. 

In [3]:
from pylabrobot.storage.inheco.scila.soap import soap_encode, soap_decode, XSI
from pylabrobot.storage.inheco.scila.scila_backend import SCILABackend


scila = SCILABackend(scila_ip="169.254.1.117")

await scila.setup()

### Status Requests

In [4]:
await scila.get_status()

'idle'

In [5]:
await scila.get_drawer_positions()

{'Drawer1': 'Closed',
 'Drawer2': 'Closed',
 'Drawer3': 'Closed',
 'Drawer4': 'Closed'}

In [6]:
drawer_indices = list(range(1, 5))

for idx in drawer_indices:
    
    temp_reponse = await scila.get_drawer_position(idx)

    print(idx, temp_reponse)

1 Closed
2 Closed
3 Closed
4 Closed


### Drawer Control

Note: By firmware, only one drawer can be opened at a time!

In [8]:
for idx in tqdm(drawer_indices):
    
    await scila.open_drawer(idx)

    await asyncio.sleep(1) # wait for 1 second
    
    await scila.close_drawer(idx)


  0%|          | 0/4 [00:00<?, ?it/s]

In [31]:
await scila.open_drawer(1)


2026-02-07 17:10:37,169 - pylabrobot.storage.inheco.scila.inheco_sila_interface - WARNING - No pending command to match response to.
2026-02-07 17:13:27,226 - pylabrobot.storage.inheco.scila.inheco_sila_interface - WARNING - No pending command to match response to.


### Temperature Control

21.01

In [23]:
await scila.set_tempeature(37.0)

In [27]:
for time_step in tqdm(range(10)):
    
    temperature_reading = await scila.get_current_temperature()

    print(temperature_reading)

    await asyncio.sleep(1)

  0%|          | 0/10 [00:00<?, ?it/s]

34.38
34.46
34.55
34.64
34.74
34.82
34.91
34.99
35.08
35.16


Uncomment the next coding cell if you want to stop temperature control:

In [ ]:
# await scila.deactivate_temperature_control()

In [1]:
from pylabrobot.arms.kx2.kx2_backend import *

In [2]:
kx2 = KX2Backend()
can = kx2.can

In [3]:
x = await kx2.initialize()
x

motor send command 1 UI 4  False (1)
motor send command 2 UI 4  False (1)
motor send command 3 UI 4  False (1)
motor send command 4 UI 4  False (1)
motor send command 6 UI 4  False (1)

axis 1
motor send command 1 UI 5  False (1)
motor send command 1 UI 6  False (1)
motor send command 1 UI 7  False (1)
motor send command 1 UI 8  False (1)
motor send command 1 UI 9  False (1)
motor send command 1 UI 10  False (1)
motor send command 1 UI 11  False (1)
motor send command 1 UI 12  False (1)
motor send command 1 UI 13  False (1)
motor send command 1 UI 14  False (1)
motor send command 1 UI 15  False (1)
motor send command 1 UI 16  False (1)
motor send command 1 UI 24  False (1)
motor send command 1 UF 1  True (2)
motor send command 1 UF 2  True (2)
motor send command 1 XM 1  False (1)
motor send command 1 XM 2  False (1)
motor send command 1 UF 3  True (2)
motor send command 1 UF 4  True (2)
motor send command 1 VH 3  False (1)
motor send command 1 VL 3  False (1)
motor send command 1 CA 45

In [18]:
await kx2.move_to_cartesian_position(
    GripperPose(
        location=Coordinate(x=-26.9701, y=160.9649, z=04.6119),
        rotation=Rotation(x=0, y=0, z=-171.05957734864037)
    ),
    vel_pct=20,
    accel_pct=20
)

ValueError: Axis <KX2Axis.ELBOW: 3> below min travel

In [10]:
await kx2.servo_gripper_open(open_position=25.0)

In [11]:
await kx2.activate_free_mode()

In [12]:
await kx2.deactivate_free_mode()

In [4]:
await kx2.servo_gripper_open(open_position=10.0)

In [14]:
home_position =  await kx2.get_cartesian_position()

home_position

GripperPose(location=Coordinate(x=-26.9701, y=160.9649, z=704.6119), rotation=Rotation(x=0, y=0, z=-171.05957734864037))

In [6]:
byonoy_dector_unit_location = await kx2.get_cartesian_position()
byonoy_dector_unit_location

GripperPose(location=Coordinate(x=-195.1244, y=518.7307, z=37.6328), rotation=Rotation(x=0, y=0, z=-161.802826074163))

In [5]:
byonoy_dector_unit_location = GripperPose(location=Coordinate(x=-195.1244, y=518.7307, z=37.6328), rotation=Rotation(x=0, y=0, z=-161.802826074163))

In [6]:
waypoint_z_0 = GripperPose(location=Coordinate(x=-195.1244, y=518.7307, z=37.6328 + 100), rotation=Rotation(x=0, y=0, z=-161.802826074163))

In [20]:
await kx2.move_to_cartesian_position(
    waypoint_z_0,
    vel_pct=20,
    accel_pct=20
)

In [19]:
await kx2.move_to_cartesian_position(
    byonoy_dector_unit_location,
    vel_pct=20,
    accel_pct=20
)

In [9]:
kx2.servo_gripper_closed_position = 8
await kx2.servo_gripper_close()

node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 MS 1  False (1)
Servo Gripper Motor Status: 1
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 CL 1  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 IQ 0  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 CL 1  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 IQ 0  False (1)


In [15]:
gp = GripperPose(location=Coordinate(x=222.8748, y=185.6677, z=520.8049), rotation=Rotation(x=0, y=0, z=-122.82045997896907))
await kx2.move_to_cartesian_position(gp, vel_pct=20, accel_pct=20)

In [10]:
drawer4 = GripperPose(location=Coordinate(x=101.001, y=296.6519, z=73.1333 + 47), rotation=Rotation(x=0, y=0, z=271.01718397191803))
await kx2.move_to_cartesian_position(drawer4, vel_pct=10, accel_pct=10)

convert_elbow_angle_to_position: 14.987054443359375 -> 161.7143894118624
MotorMoveParam(axis=<Axis.SHOULDER: 1>, position=7950488, velocity=1000, acceleration=1000, relative=False, direction=<eJointMoveDirection.ShortestWay: 3>)
MotorMoveParam(axis=<Axis.Z: 2>, position=480273, velocity=299838, acceleration=399784, relative=False, direction=<eJointMoveDirection.Normal: 0>)
MotorMoveParam(axis=<Axis.ELBOW: 3>, position=272840, velocity=1000, acceleration=1000, relative=False, direction=<eJointMoveDirection.Normal: 0>)
MotorMoveParam(axis=<Axis.WRIST: 4>, position=13190, velocity=1000, acceleration=1000, relative=False, direction=<eJointMoveDirection.ShortestWay: 3>)
motor_wait_for_move_done 4147.999458000006
motor_wait_for_move_done 3986.5379170000087
motor_wait_for_move_done 1000
motor_wait_for_move_done 1000


In [11]:
top = GripperPose(location=Coordinate(x=101.001, y=296.6519, z=300), rotation=Rotation(x=0, y=0, z=271.01718397191803))
await kx2.move_to_cartesian_position(top)

convert_elbow_angle_to_position: 14.987548828125 -> 161.718473622052
MotorMoveParam(axis=<Axis.SHOULDER: 1>, position=7950488, velocity=1000, acceleration=1000, relative=False, direction=<eJointMoveDirection.ShortestWay: 3>)
MotorMoveParam(axis=<Axis.Z: 2>, position=1199351, velocity=1695512, acceleration=3997842, relative=False, direction=<eJointMoveDirection.Normal: 0>)
MotorMoveParam(axis=<Axis.ELBOW: 3>, position=272840, velocity=1000, acceleration=1000, relative=False, direction=<eJointMoveDirection.Normal: 0>)
MotorMoveParam(axis=<Axis.WRIST: 4>, position=13190, velocity=1000, acceleration=1000, relative=False, direction=<eJointMoveDirection.ShortestWay: 3>)
motor_wait_for_move_done 1847.9994579999911
motor_wait_for_move_done 1746.921291999999
motor_wait_for_move_done 1000
motor_wait_for_move_done 1000


In [17]:
await scila.open_drawer(4)

In [18]:
await kx2.move_to_cartesian_position(drawer4, vel_pct=10, accel_pct=10)

convert_elbow_angle_to_position: 14.987548828125 -> 161.718473622052
MotorMoveParam(axis=<Axis.SHOULDER: 1>, position=7950488, velocity=1000, acceleration=1000, relative=False, direction=<eJointMoveDirection.ShortestWay: 3>)
MotorMoveParam(axis=<Axis.Z: 2>, position=480273, velocity=299838, acceleration=399784, relative=False, direction=<eJointMoveDirection.Normal: 0>)
MotorMoveParam(axis=<Axis.ELBOW: 3>, position=272840, velocity=1000, acceleration=1000, relative=False, direction=<eJointMoveDirection.Normal: 0>)
MotorMoveParam(axis=<Axis.WRIST: 4>, position=13190, velocity=1000, acceleration=1000, relative=False, direction=<eJointMoveDirection.ShortestWay: 3>)
motor_wait_for_move_done 4147.999542000005
motor_wait_for_move_done 4047.0339999999997
motor_wait_for_move_done 1000
motor_wait_for_move_done 1000


In [16]:
await kx2.servo_gripper_close()

node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 MS 1  False (1)
Servo Gripper Motor Status: 0
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 CL 1  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 IQ 0  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 CL 1  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 IQ 0  False (1)


In [20]:
await kx2.move_to_cartesian_position(top, vel_pct=10, accel_pct=10)

convert_elbow_angle_to_position: 14.987548828125 -> 161.718473622052
MotorMoveParam(axis=<Axis.SHOULDER: 1>, position=7950488, velocity=1000, acceleration=1000, relative=False, direction=<eJointMoveDirection.ShortestWay: 3>)
MotorMoveParam(axis=<Axis.Z: 2>, position=1199351, velocity=299838, acceleration=399784, relative=False, direction=<eJointMoveDirection.Normal: 0>)
MotorMoveParam(axis=<Axis.ELBOW: 3>, position=272840, velocity=1000, acceleration=1000, relative=False, direction=<eJointMoveDirection.Normal: 0>)
MotorMoveParam(axis=<Axis.WRIST: 4>, position=13190, velocity=1000, acceleration=1000, relative=False, direction=<eJointMoveDirection.ShortestWay: 3>)
motor_wait_for_move_done 4147.999582999997
motor_wait_for_move_done 4046.764290999985
motor_wait_for_move_done 1000
motor_wait_for_move_done 1000


In [ ]:
await scila.close_drawer(4)

No pending command to match response to.
No pending command to match response to.


In [39]:
await scila.open_drawer(3)

In [35]:
drawer3 = GripperPose(location=Coordinate(x=101.001, y=296.6519, z=73.1333), rotation=Rotation(x=0, y=0, z=271.01718397191803))
await kx2.move_to_cartesian_position(drawer3, vel_pct=10, accel_pct=10)

convert_elbow_angle_to_position: 14.987548828125 -> 161.718473622052
MotorMoveParam(axis=<Axis.SHOULDER: 1>, position=7950488, velocity=1000, acceleration=1000, relative=False, direction=<eJointMoveDirection.ShortestWay: 3>)
MotorMoveParam(axis=<Axis.Z: 2>, position=292375, velocity=299838, acceleration=399784, relative=False, direction=<eJointMoveDirection.Normal: 0>)
MotorMoveParam(axis=<Axis.ELBOW: 3>, position=272840, velocity=1000, acceleration=1000, relative=False, direction=<eJointMoveDirection.Normal: 0>)
MotorMoveParam(axis=<Axis.WRIST: 4>, position=13190, velocity=1000, acceleration=1000, relative=False, direction=<eJointMoveDirection.ShortestWay: 3>)
motor_wait_for_move_done 4774.999332999985
motor_wait_for_move_done 4673.443916000018
motor_wait_for_move_done 1000
motor_wait_for_move_done 1000


In [36]:
await kx2.servo_gripper_open()

convert_elbow_angle_to_position: 14.987548828125 -> 161.718473622052
MotorMoveParam(axis=<Axis.SERVO_GRIPPER: 6>, position=79577, velocity=77970, acceleration=636620, relative=False, direction=<eJointMoveDirection.Normal: 0>)
motor_wait_for_move_done 1244.9996249999967


In [37]:
await kx2.move_to_cartesian_position(top, vel_pct=10, accel_pct=10)

convert_elbow_angle_to_position: 14.987548828125 -> 161.718473622052
MotorMoveParam(axis=<Axis.SHOULDER: 1>, position=7950488, velocity=1000, acceleration=1000, relative=False, direction=<eJointMoveDirection.ShortestWay: 3>)
MotorMoveParam(axis=<Axis.Z: 2>, position=1199351, velocity=299838, acceleration=399784, relative=False, direction=<eJointMoveDirection.Normal: 0>)
MotorMoveParam(axis=<Axis.ELBOW: 3>, position=272840, velocity=1000, acceleration=1000, relative=False, direction=<eJointMoveDirection.Normal: 0>)
MotorMoveParam(axis=<Axis.WRIST: 4>, position=13190, velocity=1, acceleration=3, relative=False, direction=<eJointMoveDirection.ShortestWay: 3>)
motor_wait_for_move_done 4774.999125000031
motor_wait_for_move_done 4673.237708000001
motor_wait_for_move_done 1000
motor_wait_for_move_done 1000


In [40]:
await scila.close_drawer(3)